일단 시피유로 인퍼런스 ㄱㄱ
1. 모델 불러오기 (나중에는 항상 메모리에 올려놓기 ㄱㄱ)
2. 데이터 단일 혹은 배치단위로 읽어오기
    - 일단 npy
    - 나중에 xml을 받아야함..

In [ ]:
### json style ###

CFG = {
    'DEVICE' : 'cpu',
    'TARGET_COLS' : [
        'ECG_event_3d_AFIB_AFL-keyword_v2_onset',
        'ECG_event_3d_CIA-keyword_v2_onset',
        'ECG_event_7d_AFIB_AFL-keyword_v2_onset',
        'ECG_event_7d_CIA-keyword_v2_onset',
        'ECG_event_14d_AFIB_AFL-keyword_v2_onset',
        'ECG_event_14d_CIA-keyword_v2_onset',
        'ECG_event_30d_AFIB_AFL-keyword_v2_onset',
        'ECG_event_30d_CIA-keyword_v2_onset',
        'ECG_event_90d_AFIB_AFL-keyword_v2_onset',
        'ECG_event_90d_CIA-keyword_v2_onset',
        'AFIB_AFL-keyword_v2',
        'CIA-keyword_v2'
    ],
    'MODEL_INFO' : {
        'config':{
            'embed_dim':768,  
            'patch_size':32, 
            'seq_length':2560,
            'in_channels':12, 
            'encoder':'vit_encoder',
            'merge_mode':'projection',  
            'num_classes':12,
        },        
        'model_path_list': [
            '/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_AUPRC-Ep_3-Lo_0.082-M0_0.793-M1_0.494.pth',
            '/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_AUROC-Ep_7-Lo_0.082-M0_0.800-M1_0.493.pth',
            '/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_loss-Ep_5-Lo_0.082-M0_0.797-M1_0.493.pth',
        ],
    }
}


: 

In [ ]:
from model_architecture import SynAI
import torch

def model_build(model_info):
    model = SynAI.MAE_1D_250409_v3(
        seq_length  = model_info['seq_length'],
        in_channels = model_info['in_channels'],
        patch_size  = model_info['patch_size'],
        embed_dim   = model_info['embed_dim'],
        merge_mode  = model_info['merge_mode'],  # linear_projection avg add
        encoder     = model_info['encoder'],
    )
    model = SynAI.OnlyEncoderForFT_250409(
        model,
        num_classes = model_info['num_classes'],
        embed_dim = model_info['embed_dim'],
    )
    return model

def load_models(model_path_list, model_info, device='cpu'):
    models = []
    for model_path in model_path_list:
        checkpoint = torch.load(model_path, weights_only=True)
        model = model_build(model_info).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])
        models.append({'model_path': model_path, 'model': model})
    return models



In [ ]:
models = load_models(CFG['MODEL_INFO']['model_path_list'], 
                     CFG['MODEL_INFO']['config'], device=CFG['DEVICE'])


In [ ]:
from torch.utils.data import DataLoader
from processing import transforms, dataset

transform_pipeline = {}
transform_pipeline['val'] = transforms.TransformPipeline([
    transforms.NormalizeECG(method="tanh", scope="lead-wise", scale=1),
])


In [ ]:
import pandas as pd
df_path = '/mnt/home/Datasets_processed/EKG_Latest/processed_metadata_v6-sv2_subset_2_holdout.csv'
df = pd.read_csv(df_path)
tmp_npy_path = df['bpf0540_or3-npy_path'][:2]
tmp_npy_path.to_list()

In [ ]:
bandpass_info = {
    "lowcut": 0.5,
    "highcut": 40.0,
    "order": 3,
}

In [ ]:
import numpy as np
import torch
from processing import transforms, dataset

# 배치 데이터를 저장할 리스트
batch_ecg = []

for i in range(len(tmp_npy_path)):
    # ECG 데이터 로드 및 전처리
    tmp_ecg = np.load('/mnt' + tmp_npy_path[i])
    tmp_ecg = dataset.ECGDataset_v2.preprocess_ecg(
        x               = tmp_ecg, 
        scaling_factor  = None,
        do_bandpass     = bandpass_info,
        original_fs     = 500,
        target_fs       = 256,
        do_normalize    = False,
        transforms      = transform_pipeline['val']
    )
    # 배치에 추가
    batch_ecg.append(tmp_ecg)

# 리스트를 PyTorch 텐서로 변환 (배치 크기, 채널, 시퀀스 길이)
batch_ecg_tensor = torch.tensor(np.stack(batch_ecg), dtype=torch.float32)

# 결과 확인
print("배치 텐서 크기:", batch_ecg_tensor.shape)  # 예: (배치 크기, 채널, 시퀀스 길이)

In [ ]:
import torch
from typing import Dict, Optional, List
from torch.utils.data import DataLoader
from tqdm import tqdm

@torch.no_grad()
def inference_tensor(
    model: torch.nn.Module,
    input_tensor: torch.Tensor,
    activation: Optional[str] = 'sigmoid',
    to_cpu: bool = True
) -> Dict[str, torch.Tensor]:
    """
    단일 샘플 또는 배치 텐서에 대해 추론 결과 반환
    """
    original_input = input_tensor.clone().detach()
    device = next(model.parameters()).device
    model.eval()
    inference_input = input_tensor.to(device)
    is_batch = inference_input.dim() >= 3
    if not is_batch:
        inference_input = inference_input.unsqueeze(0)
    with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
        logits = model(inference_input)
    if not is_batch:
        logits = logits.squeeze(0)
    probs = None
    if activation == 'sigmoid':
        probs = torch.sigmoid(logits)
    elif activation == 'softmax':
        probs = torch.softmax(logits, dim=-1)
    if to_cpu:
        logits = logits.cpu()
        if probs is not None:
            probs = probs.cpu()
    result = {
        "input": original_input,
        "logits": logits
    }
    if probs is not None:
        result["probs"] = probs
    return result


In [ ]:
result = inference_tensor(
    models[0]['model'], batch_ecg_tensor)
result

In [ ]:
result['probs'].shape

In [ ]:
af_idx = CFG['TARGET_COLS'].index('ECG_event_14d_AFIB_AFL-keyword_v2_onset')
cia_idx = CFG['TARGET_COLS'].index('ECG_event_14d_CIA-keyword_v2_onset')

In [ ]:
result['probs'][:, af_idx]
result['probs'][:, cia_idx]

In [ ]:
import joblib
af_cal_model = joblib.load('/mnt/home/bgk/macai-engine-v1/trained_models/v1.2.1/calibration_model-AF.pkl')
af_prob_cal = af_cal_model.predict_proba(result['probs'][:, af_idx].reshape(-1, 1))[:, 1]
cia_cal_model = joblib.load('/mnt/home/bgk/macai-engine-v1/trained_models/v1.2.1/calibration_model-CIA.pkl')
cia_prob_cal = cia_cal_model.predict_proba(result['probs'][:, cia_idx].reshape(-1, 1))[:, 1]

In [ ]:
af_high_g_thr = 0.18594001505695362
af_inter_g_thr = 0.09283757956391994
cia_high_g_thr = 0.6134156782179706
cia_inter_g_thr = 0.39689483565475614

In [ ]:
def classify_risk(
    probs:np.array, high_thr:float, inter_thr:float, method:str=None):
    '''
        probs: 위험도 확률 리스트
        high_thr: 고위험군 기준 임계값 (상위 10%)
        inter_thr: 중간위험군 기준 임계값 (상위 10% ~ 상위 30%)
    '''
    results = []    
    if method == 'mean':
        probs = probs.mean()
        if probs >= high_thr:
            return [probs, "High"]
        elif probs >= inter_thr:
            return [probs, "Intermediate"]
        else:
            return [probs, "Low"]
    else:
        for prob in probs:
            if prob >= high_thr:
                results.append([prob, "High"])
            elif prob >= inter_thr:
                results.append([prob, "Intermediate"])
            else:
                results.append([prob, "Low"])
        return results

In [ ]:
af_risk = classify_risk(af_prob_cal, af_high_g_thr, af_inter_g_thr, method='mean')
cia_risk = classify_risk(cia_prob_cal, cia_high_g_thr, cia_inter_g_thr, method='mean')
af_risk, cia_risk

## yaml test

In [ ]:
import yaml

config_path = '/mnt/home/bgk/macai-engine-v1/trained_models/v1.2.1/config.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

print(config)

In [ ]:
from processing import infer
models = infer.load_models(config['MODEL_INFO']['MODEL_PATHS'], 
                           config['MODEL_INFO']['CONFIG'], device=config['MODEL_INFO']['DEVICE'])

In [ ]:
import joblib
af_cal_model = joblib.load(config['MODEL_INFO']['AF_CAL_MODEL_PATH'])
cia_cal_model = joblib.load(config['MODEL_INFO']['CIA_CAL_MODEL_PATH'])

In [ ]:
from torch.utils.data import DataLoader
from processing import transforms, dataset, infer

transform_pipeline = {}
transform_pipeline['val'] = transforms.TransformPipeline([
    transforms.NormalizeECG(method="tanh", scope="lead-wise", scale=1),
])


### 임시 ###
import numpy as np
import torch

batch_ecg = []
for tmp_npy_path in config['DATA_INFO']['ECG_PATHS']:
    tmp_ecg = np.load(tmp_npy_path)
    tmp_ecg = dataset.ECGDataset_v2.preprocess_ecg(
        x               = tmp_ecg, 
        scaling_factor  = config['DATA_INFO']['PREPROCESS']['scaling_factor'],
        do_bandpass     = config['DATA_INFO']['PREPROCESS']['do_bandpass'],
        original_fs     = config['DATA_INFO']['PREPROCESS']['original_fs'],
        target_fs       = config['DATA_INFO']['PREPROCESS']['target_fs'],
        do_normalize    = config['DATA_INFO']['PREPROCESS']['do_normalize'],
        transforms      = transform_pipeline['val']
    )
    batch_ecg.append(tmp_ecg)
batch_ecg_tensor = torch.tensor(np.stack(batch_ecg), dtype=torch.float32)
print(batch_ecg_tensor.shape)

In [ ]:
### 임시 ###
results = []
for i in range(len(models)):
    result = infer.inference_tensor(models[i]['model'], batch_ecg_tensor)
    results.append(result)
probs_list = [result['probs'] for result in results]  # results는 각 모델의 결과 리스트
stacked_probs = torch.stack(probs_list, dim=0)  # (모델 개수, 배치 크기, 클래스 개수)
mean_probs = stacked_probs.mean(dim=0)  # 모델 차원(0)을 기준으로 평균 계산
mean_probs = np.array(mean_probs)
print("평균 확률값:", mean_probs.shape)

In [ ]:
af_idx = config['DATA_INFO']['OUTPUT_CLS_INFO']['cls_names'].index(config['DATA_INFO']['OUTPUT_CLS_INFO']['target_cls']['AF'])
cia_idx = config['DATA_INFO']['OUTPUT_CLS_INFO']['cls_names'].index(config['DATA_INFO']['OUTPUT_CLS_INFO']['target_cls']['CIA'])
af_prob_cal = af_cal_model.predict_proba(mean_probs[:, af_idx].reshape(-1, 1))[:, 1]
cia_prob_cal = cia_cal_model.predict_proba(mean_probs[:, cia_idx].reshape(-1, 1))[:, 1]

In [ ]:
af_risk = infer.classify_risk(
    probs = af_prob_cal, 
    high_thr = config['MODEL_INFO']['RISK_THRESHOLD']['af_high_g_thr'], 
    inter_thr = config['MODEL_INFO']['RISK_THRESHOLD']['af_inter_g_thr'], 
    method = 'mean'
)
cia_risk = infer.classify_risk(
    probs = cia_prob_cal, 
    high_thr = config['MODEL_INFO']['RISK_THRESHOLD']['cia_high_g_thr'], 
    inter_thr = config['MODEL_INFO']['RISK_THRESHOLD']['cia_inter_g_thr'], 
    method = 'mean'
)
af_risk, cia_risk

## xml api test

In [ ]:
import os
import requests
import argparse
import numpy as np
import pandas as pd
from glob import glob

def send_xml_to_api(filepath: str, api_url: str) -> dict:
    filename = os.path.basename(filepath)
    with open(filepath, mode='rb') as f:
        files = {'file': (filename, f, 'application/xml')}
        response = requests.post(api_url, files=files)
        print (response.json().keys())
        if response.status_code == 200:
            return response.json()
        else:
            print (True)
            response.raise_for_status()

def parse_api_response(response: dict) -> tuple:
    message = response['message']
    status  = response['status']
    metadata = response['data']['metadata']
    signal  = response['data']['signal']
    return message, status, metadata, signal


In [ ]:
class args:
    api_url = 'http://localhost:18002/parse/parse_xml'

In [ ]:
# xml_filepath = '/mnt/home/bgk/macai-engine-v1/data/samples/xmls/10144268_2024-06-02_2024060214062189_2024060214053964.xml'
# xml_filepath = '/mnt/home/bgk/macai-engine-v1/data/samples/xmls/10203160_2023-10-04_2023100411441625_2023101110031840.xml'
xml_filepath = '/mnt/home/bgk/macai-engine-v1/data/samples/xmls/10302870_2023-10-17_2023101716092374_2023101716095795.xml'
response = send_xml_to_api(xml_filepath, args.api_url)

In [ ]:
message, status, metadata, signal = parse_api_response(response)

In [ ]:
import pickle

data_to_save = {
    'signal': signal,
    'metadata': metadata
}

# 저장할 파일 경로
output_filepath = '/mnt/home/bgk/macai-engine-v1/data/samples/pkl/10302870_2023-10-17_2023101716092374_2023101716095795.pkl'


# --- 2. Pickle 파일로 저장하기 ---
# 'wb'는 "write binary"의 약자로, 바이너리 쓰기 모드를 의미합니다.
with open(output_filepath, 'wb') as f:
    pickle.dump(data_to_save, f)


In [ ]:
with open(output_filepath, 'rb') as f:
    loaded_data = pickle.load(f)

print("\n파일에서 불러온 데이터:")

# --- 4. 불러온 데이터 확인 ---
loaded_signal = loaded_data['signal']
loaded_metadata = loaded_data['metadata']
loaded_metadata

In [ ]:
message, status

In [ ]:
metadata

In [ ]:
statement = str(metadata['statement'])
metadata['statement'] = statement
metadata

In [ ]:
signal_seq = [key for key in signal.keys()]
signal_arr = np.array([signal[key] for key in signal.keys()]).astype(np.float32)
print(signal_seq)
print(signal_arr.shape)
signal

In [ ]:
signal_arr.dtype

In [ ]:
metadata

In [ ]:
            response = send_xml_to_api(xml_filepath, args.api_url)
            message, status, metadata, signal = parse_api_response(response)


In [ ]:
    parser.add_argument('--file', type=str, required=False, default='')
    parser.add_argument('--directory', type=str, required=False, default='./xml')
    parser.add_argument('--api_url', type=str, required=False, default='http://localhost:18002/parse/parse_xml')
    parser.add_argument('--signal_dir', type=str, required=False, default='./npy')
    parser.add_argument('--output_path', type=str, required=False, default='./output.csv')
    args = parser.parse_args()


# TEST

In [3]:
import yaml

config_path = '/mnt/home/bgk/macai-engine-v1/trained_models/v1.3.0/config.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

print(config)

{'MODEL_INFO': {'DEVICE': 'cpu', 'CONFIG': {'embed_dim': 768, 'patch_size': 32, 'seq_length': 2560, 'in_channels': 12, 'encoder': 'vit_encoder', 'merge_mode': 'projection', 'num_classes': 12}, 'MODEL_PATHS': ['/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_AUPRC-Ep_3-Lo_0.082-M0_0.793-M1_0.494.pth', '/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_AUROC-Ep_7-Lo_0.082-M0_0.800-M1_0.493.pth', '/mnt/home/bgk/macai-model-experimental/_kimbg_code/SynAI_v2/outputs/checkpoint/CL_step3-TRN-MacAI_v1_2-0819 copy 21/0fold/Best_loss-Ep_5-Lo_0.082-M0_0.797-M1_0.493.pth'], 'AF_CAL_MODEL_PATH': '/mnt/home/bgk/macai-engine-v1/trained_models/v1.3.0/calibration_model-AF.pkl', 'CIA_CAL_MODEL_PATH': '/mnt/home/bgk/macai-engine-v1/trained_models/v1.3.0/calibration_model-CIA.pkl', 'OUTPUT_CLS_INFO': {'cls_names': ['ECG_event_3d_AFIB_AFL-keyword_v2_ons

In [4]:
from processing import infer
models = infer.load_models(config['MODEL_INFO']['MODEL_PATHS'], 
                           config['MODEL_INFO']['CONFIG'], device=config['MODEL_INFO']['DEVICE'])
import joblib
af_cal_model = joblib.load(config['MODEL_INFO']['AF_CAL_MODEL_PATH'])
cia_cal_model = joblib.load(config['MODEL_INFO']['CIA_CAL_MODEL_PATH'])

KeyboardInterrupt: 

In [8]:
from parsing import parse_xml 
import numpy as np
# api_url = 'http://localhost:18002/parse/parse_xml'
api_url = 'http://localhost:18003/parse/parse_xml'
response = parse_xml.send_xml_to_api(config['DATA_INFO']['XML_INFO']['xml_paths'][0], api_url)
message, status, metadata, signal = parse_xml.parse_api_response(response)
statement = str(metadata['statement'])
metadata['statement'] = statement

signal_seq = [key for key in signal.keys()]
signal_arr = np.array([signal[key] for key in signal.keys()]).astype(np.float32)
print(signal_seq)
print(signal_arr.shape)

ConnectionError: HTTPConnectionPool(host='localhost', port=18003): Max retries exceeded with url: /parse/parse_xml (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9b619458a0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
from processing import transforms

transform_pipeline = []
for transform_info in config['DATA_INFO']['TRANSFORMS']['val']:
    key = transform_info['type']
    params = {k: v for k, v in transform_info.items() if k != 'type'}
    transform_pipeline.append(getattr(transforms, key.split('(')[0])(**params))
transform_pipeline = transforms.TransformPipeline(transform_pipeline)
transform_pipeline

In [ ]:
from _dev import dataset

signal = dataset.ECGDataset.load_ecg_from_xml(
    config['DATA_INFO']['XML_INFO']['xml_paths'][0],
    do_bandpass=config['DATA_INFO']['PREPROCESS']['do_bandpass'],
    api_url=api_url,
)
signal.shape

In [ ]:
signal = dataset.ECGDataset.preprocess_ecg(
    signal,
    target_fs=config['DATA_INFO']['PREPROCESS']['target_fs'],
    do_normalize=config['DATA_INFO']['PREPROCESS']['do_normalize'],
    transforms=transform_pipeline,
)
signal.shape

In [ ]:
import torch

batch_ecg_tensor = []
for path in config['DATA_INFO']['XML_INFO']['xml_paths']:
    signal = dataset.ECGDataset.load_ecg_from_xml(
        fpath=path,
        do_bandpass=config['DATA_INFO']['PREPROCESS']['do_bandpass'],
        api_url=api_url)
    if signal is not None:
        signal = dataset.ECGDataset.preprocess_ecg(
            signal,
            target_fs=config['DATA_INFO']['PREPROCESS']['target_fs'],
            do_normalize=config['DATA_INFO']['PREPROCESS']['do_normalize'],
            transforms=transform_pipeline,
        )        
        batch_ecg_tensor.append(signal)
batch_ecg_tensor = torch.tensor(np.stack(batch_ecg_tensor), dtype=torch.float32)
batch_ecg_tensor.shape

In [ ]:
### 임시 ###
results = []
for i in range(len(models)):
    result = infer.inference_tensor(models[i]['model'], batch_ecg_tensor)
    results.append(result)
probs_list = [result['probs'] for result in results]  # results는 각 모델의 결과 리스트
stacked_probs = torch.stack(probs_list, dim=0)  # (모델 개수, 배치 크기, 클래스 개수)
mean_probs = stacked_probs.mean(dim=0)  # 모델 차원(0)을 기준으로 평균 계산
mean_probs = np.array(mean_probs)
print("평균 확률값:", mean_probs.shape)

In [ ]:
af_idx = config['MODEL_INFO']['OUTPUT_CLS_INFO']['cls_names'].index(config['MODEL_INFO']['OUTPUT_CLS_INFO']['target_cls']['AF'])
cia_idx = config['MODEL_INFO']['OUTPUT_CLS_INFO']['cls_names'].index(config['MODEL_INFO']['OUTPUT_CLS_INFO']['target_cls']['CIA'])
print(af_idx, cia_idx)
af_prob_cal = af_cal_model.predict_proba(mean_probs[:, af_idx].reshape(-1, 1))[:, 1]
cia_prob_cal = cia_cal_model.predict_proba(mean_probs[:, cia_idx].reshape(-1, 1))[:, 1]

In [ ]:
af_risk = infer.classify_risk(
    probs = af_prob_cal, 
    high_thr = config['MODEL_INFO']['RISK_THRESHOLD']['af_high_g_thr'], 
    inter_thr = config['MODEL_INFO']['RISK_THRESHOLD']['af_inter_g_thr'], 
    method = 'mean'
)
cia_risk = infer.classify_risk(
    probs = cia_prob_cal, 
    high_thr = config['MODEL_INFO']['RISK_THRESHOLD']['cia_high_g_thr'], 
    inter_thr = config['MODEL_INFO']['RISK_THRESHOLD']['cia_inter_g_thr'], 
    method = 'mean'
)
af_risk, cia_risk

In [ ]:
af_prob_cal

In [ ]:
af_prob_cal

In [ ]:
af_prob_cal

In [ ]:
from _dev import tmp

In [ ]:
import _dev

In [ ]:
_dev.tmp

In [ ]:
import importlib
importlib.reload(_dev)


In [ ]:
from _dev import dataset
dataset.tmp

In [ ]:
import importlib
# _dev.dataset 모듈을 dev_dataset 이라는 다른 이름으로 불러옵니다.
from _dev import dataset as dev_dataset

# 이제 dev_dataset을 reload 해야 합니다.
importlib.reload(dev_dataset)

# dev_dataset을 통해 tmp 함수를 호출합니다.
dev_dataset.tmp()

In [ ]:
print(f"현재 로드된 파일 경로: {dev_dataset.__file__}")


In [ ]:
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
project_root

# TEST v2

In [7]:
import sys
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="TypedStorage is deprecated.")

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print('project_root :', project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)


import yaml
config_path = '/mnt/home/bgk/macai-engine-v1/trained_models/v1.3.0/config.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)


project_root : /mnt/home/bgk/macai-engine-v1


In [8]:
import requests
response = requests.get(config['FOR_INFER_ENGINE']['XML_HEALTHCHECK_URL'])
print(response.json())

{'success': True, 'message': 'OK'}


In [9]:
from src.processing import infer
import joblib
models = infer.load_models(config['MODEL_INFO']['MODEL_PATHS'], 
                           config['MODEL_INFO']['CONFIG'], device=config['MODEL_INFO']['DEVICE'])
af_cal_model = joblib.load(config['MODEL_INFO']['AF_CAL_MODEL_PATH'])
cia_cal_model = joblib.load(config['MODEL_INFO']['CIA_CAL_MODEL_PATH'])


from src.processing import transforms
transform_pipeline = []
for transform_info in config['PREPROCESS']['transforms']['val']:
    key = transform_info['type']
    params = {k: v for k, v in transform_info.items() if k != 'type'}
    transform_pipeline.append(getattr(transforms, key.split('(')[0])(**params))
transform_pipeline = transforms.TransformPipeline(transform_pipeline)

In [11]:
import torch
import numpy as np
from src._dev import dataset
batch_ecg_tensor = []
for path in config['FOR_INFER_ENGINE']['XML_PATHS']:
    signal = dataset.ECGDataset.load_ecg_from_xml(
        fpath=path,
        do_bandpass=config['PREPROCESS']['do_bandpass'],
        api_url=config['FOR_INFER_ENGINE']['XML_PARSER_URL']
    )
    if signal is not None:
        signal = dataset.ECGDataset.preprocess_ecg(
            signal,
            target_fs=config['PREPROCESS']['target_fs'],
            do_normalize=config['PREPROCESS']['do_normalize'],
            transforms=transform_pipeline,
        )        
        batch_ecg_tensor.append(signal)
batch_ecg_tensor = torch.tensor(np.stack(batch_ecg_tensor), dtype=torch.float32)


results = []
for i in range(len(models)):
    result = infer.inference_tensor(models[i]['model'], batch_ecg_tensor)
    results.append(result)
probs_list = [result['probs'] for result in results]  # results는 각 모델의 결과 리스트
stacked_probs = torch.stack(probs_list, dim=0)  # (모델 개수, 배치 크기, 클래스 개수)
mean_probs = stacked_probs.mean(dim=0)  # 모델 차원(0)을 기준으로 평균 계산
mean_probs = np.array(mean_probs)


af_idx = config['MODEL_INFO']['OUTPUT_CLS_INFO']['cls_names'].index(config['MODEL_INFO']['OUTPUT_CLS_INFO']['target_cls']['AF'])
cia_idx = config['MODEL_INFO']['OUTPUT_CLS_INFO']['cls_names'].index(config['MODEL_INFO']['OUTPUT_CLS_INFO']['target_cls']['CIA'])
af_prob_cal = af_cal_model.predict_proba(mean_probs[:, af_idx].reshape(-1, 1))[:, 1]
cia_prob_cal = cia_cal_model.predict_proba(mean_probs[:, cia_idx].reshape(-1, 1))[:, 1]


af_risk = infer.classify_risk(
    probs = af_prob_cal, 
    high_thr = config['MODEL_INFO']['RISK_THRESHOLD']['af_high_g_thr'], 
    inter_thr = config['MODEL_INFO']['RISK_THRESHOLD']['af_inter_g_thr'], 
    method = 'mean'
)
cia_risk = infer.classify_risk(
    probs = cia_prob_cal, 
    high_thr = config['MODEL_INFO']['RISK_THRESHOLD']['cia_high_g_thr'], 
    inter_thr = config['MODEL_INFO']['RISK_THRESHOLD']['cia_inter_g_thr'], 
    method = 'mean'
)

print(af_risk, cia_risk)

[0.06427235352070447, 'Low'] [0.2090382394771072, 'Low']
